In [ ]:
# 셀 1: 함수 선언 및 import
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import sklearn
import xgboost
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor

# 시계열 교차검증을 위한 sklearn 함수들
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error

import random as rn
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

RANDOM_SEED = 2024
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)

def smape(gt, preds):
    """SMAPE 계산 함수"""
    gt = np.array(gt)
    preds = np.array(preds)
    v = 2 * abs(preds - gt) / (abs(preds) + abs(gt) + 1e-9)  # 0 division 방지
    score = np.mean(v) * 100
    return score

def weighted_mse(alpha=1):
    """XGBoost용 가중치 MSE 손실함수"""
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual > 0, -2 * alpha * residual, -2 * residual)
        hess = np.where(residual > 0, 2 * alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

def custom_smape(preds, dtrain):
    """XGBoost용 사용자 정의 SMAPE 평가함수"""
    labels = dtrain.get_label()
    return 'custom_smape', np.mean(2 * abs(preds - labels) / (abs(preds) + abs(labels) + 1e-9)) * 100

def pseudo_huber_loss(delta=1.0):
    """로버스트 Pseudo-Huber 손실함수 (선택사항)"""
    def pseudo_huber_objective(label, pred):
        residual = label - pred
        scale = delta
        grad = residual / np.sqrt(1 + (residual / scale) ** 2)
        hess = 1 / (1 + (residual / scale) ** 2) ** (3/2)
        return grad, hess
    return pseudo_huber_objective

In [ ]:
# 셀 2: 데이터 불러오기 및 기본 전처리
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
building_info = pd.read_csv('building_info.csv')

# 컬럼명 영문 변환
train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
if 'num_date_time' in train.columns:
    train.drop('num_date_time', axis=1, inplace=True)

test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
if 'num_date_time' in test.columns:
    test.drop('num_date_time', axis=1, inplace=True)

building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

# 건물유형 영문 번역
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}

building_info['building_type'] = building_info['building_type'].replace(translation_dict)

# 태양광/ESS 활용 여부 플래그 생성
building_info['solar_power_utility'] = np.where(building_info.solar_power_capacity != '-', 1, 0)
building_info['ess_utility'] = np.where(building_info.ess_capacity != '-', 1, 0)

# 건물 정보 병합
train = pd.merge(train, building_info, on='building_number', how='left')
test = pd.merge(test, building_info, on='building_number', how='left')

# 냉방면적비 추가
train["cooling_ratio"] = (
    train["cooling_area"] / train["total_area"]
).replace([np.inf, -np.inf], np.nan).fillna(0)

test["cooling_ratio"] = (
    test["cooling_area"] / test["total_area"]
).replace([np.inf, -np.inf], np.nan).fillna(0)

# 결측치 보간 (선형보간 사용)
train['windspeed'] = train['windspeed'].interpolate(method='linear')
train['humidity'] = train['humidity'].interpolate(method='linear')
test['windspeed'] = test['windspeed'].interpolate(method='linear')
test['humidity'] = test['humidity'].interpolate(method='linear')

print("데이터 기본 전처리 완료")
print(f"Train shape: {train.shape}, Test shape: {test.shape}")
print(f"Building types: {train.building_type.unique()}")

In [ ]:
# 셀 3: 날짜/시간 피처 생성 (포리에 변환 포함)
train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')
test['date_time'] = pd.to_datetime(test['date_time'], format='%Y%m%d %H')

# 기본 시간 피처
for df in [train, test]:
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df['day_of_week'] = df['date_time'].dt.dayofweek  # 0=월요일, 6=일요일
    df['day_of_year'] = df['date_time'].dt.dayofyear
    df['week_of_year'] = df['date_time'].dt.isocalendar().week

# *** 개선사항 1: 포리에 변환 (다중 계절성) ***
for df in [train, test]:
    # 시간별 계절성 (24시간 주기)
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 24.0)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 24.0)
    
    # 일별 계절성 (7일 주기)
    df['sin_day_of_week'] = np.sin(2 * np.pi * df['day_of_week'] / 7.0)
    df['cos_day_of_week'] = np.cos(2 * np.pi * df['day_of_week'] / 7.0)
    
    # 월별 계절성 (12개월 주기)
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12.0)
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12.0)
    
    # 연간 계절성 (365일 주기)
    df['sin_day_of_year'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
    df['cos_day_of_year'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)

# *** 개선사항 2: 휴일 및 특수일 플래그 ***
# 한국 공휴일
korean_holidays = [
    '2024-06-06',  # 현충일 (목)
    '2024-08-15'   # 광복절 (목)
]

for df in [train, test]:
    # 기본 휴일 (주말 + 공휴일)
    df['is_holiday'] = np.where(
        (df['day_of_week'] >= 5) | 
        (df['date_time'].dt.strftime('%Y-%m-%d').isin(korean_holidays)), 
        1, 0
    )
    
    # 주말 vs 평일
    df['is_weekend'] = np.where(df['day_of_week'] >= 5, 1, 0)
    
    # 시간대별 구분
    df['time_period'] = pd.cut(df['hour'], 
                              bins=[-1, 6, 12, 18, 24], 
                              labels=['night', 'morning', 'afternoon', 'evening'])

# 대형마트 의무휴업 일요일
mart_closure_sundays = [
    '2024-06-09', '2024-06-23',  # 6월
    '2024-07-14', '2024-07-28',  # 7월
    '2024-08-11', '2024-08-25'   # 8월
]

for df in [train, test]:
    df['mart_closure_sunday'] = np.where(
        (df['day_of_week'] == 6) & 
        (df['date_time'].dt.strftime('%Y-%m-%d').isin(mart_closure_sundays)), 
        1, 0
    )

print("시간 피처 및 포리에 변환 완료")
print("생성된 시간 피처들:", [col for col in train.columns if any(x in col for x in ['sin_', 'cos_', 'is_', 'time_'])])

In [ ]:
# 셀 4: 날씨 파생 피처 및 냉난방도시간(CDH/HDH) 생성

def calculate_day_values(dataframe, target_column, output_column, aggregation_func):
    """일별 통계값 계산 함수"""
    result_dict = {}
    grouped = dataframe.groupby(['building_number', 'month', 'day'])[target_column].agg(aggregation_func)
    
    for (building, month, day), value in grouped.items():
        result_dict.setdefault(building, {}).setdefault(month, {})[day] = value
    
    dataframe[output_column] = [
        result_dict.get(row['building_number'], {}).get(row['month'], {}).get(row['day'], None)
        for _, row in dataframe.iterrows()
    ]

# *** 개선사항 3: 날씨 변수 파생 피처 강화 ***
for df in [train, test]:
    # 일별 온도 통계
    calculate_day_values(df, 'temperature', 'day_max_temperature', 'max')
    calculate_day_values(df, 'temperature', 'day_mean_temperature', 'mean')
    calculate_day_values(df, 'temperature', 'day_min_temperature', 'min')
    
    # 일교차
    df['day_temperature_range'] = df['day_max_temperature'] - df['day_min_temperature']
    
    # THI (불쾌지수) - 더 정확한 공식
    df['THI'] = 9/5 * df['temperature'] - 0.55 * (1 - df['humidity']/100) * (9/5 * df['temperature'] - 26) + 32
    
    # WCT (체감온도) - 풍속 고려
    df['WCT'] = 13.12 + 0.6125 * df['temperature'] - 11.37 * (df['windspeed']**0.16) + \
                0.3965 * (df['windspeed']**0.16) * df['temperature']
    
    # 습구온도 근사치
    df['wet_bulb_temp'] = df['temperature'] * np.arctan(0.151977 * np.sqrt(df['humidity'] + 8.313659)) + \
                         np.arctan(df['temperature'] + df['humidity']) - \
                         np.arctan(df['humidity'] - 1.676331) + \
                         0.00391838 * (df['humidity']**1.5) * np.arctan(0.023101 * df['humidity']) - 4.686035
    
    # 절대습도 계산
    df['absolute_humidity'] = (6.112 * np.exp((17.67 * df['temperature'])/(df['temperature'] + 243.5)) * 
                              df['humidity'] * 2.1674) / (273.15 + df['temperature'])

# *** 개선사항 4: CDH/HDH (냉난방도시간) 개선 ***
def enhanced_CDH_HDH(temperature_series, cooling_base=26, heating_base=18):
    """개선된 냉난방도시간 계산 (11시간 슬라이딩 윈도우)"""
    temp = np.array(temperature_series)
    
    # CDH (냉방도시간)
    cdh_values = np.maximum(0, temp - cooling_base)
    cdh_cumsum = np.cumsum(cdh_values)
    cdh_result = np.concatenate([
        cdh_cumsum[:11], 
        cdh_cumsum[11:] - cdh_cumsum[:-11]
    ])
    
    # HDH (난방도시간) 
    hdh_values = np.maximum(0, heating_base - temp)
    hdh_cumsum = np.cumsum(hdh_values)
    hdh_result = np.concatenate([
        hdh_cumsum[:11],
        hdh_cumsum[11:] - hdh_cumsum[:-11]
    ])
    
    return cdh_result, hdh_result

# 건물별 CDH/HDH 계산
def calculate_and_add_cdh_hdh(dataframe):
    cdhs, hdhs = [], []
    for building_num in range(1, 101):
        building_data = dataframe[dataframe['building_number'] == building_num]
        if len(building_data) > 0:
            temp_values = building_data['temperature'].values
            cdh, hdh = enhanced_CDH_HDH(temp_values)
            cdhs.extend(cdh)
            hdhs.extend(hdh)
    return cdhs, hdhs

# CDH/HDH 계산 및 추가
train_cdh, train_hdh = calculate_and_add_cdh_hdh(train)
train['CDH'] = train_cdh
train['HDH'] = train_hdh

test_cdh, test_hdh = calculate_and_add_cdh_hdh(test)
test['CDH'] = test_cdh  
test['HDH'] = test_hdh

print("날씨 파생 피처 및 CDH/HDH 계산 완료")
print("추가된 날씨 피처:", ['day_temperature_range', 'THI', 'WCT', 'wet_bulb_temp', 'absolute_humidity', 'CDH', 'HDH'])

In [ ]:
# 셀 5: 과거 부하 래그 피처 생성 (핵심 개선사항)

def create_lag_features(df, target_col='power_consumption', building_col='building_number'):
    """
    시계열 래그 피처 생성
    - 1시간, 24시간(일주기), 168시간(주주기) 래그
    - 롤링 평균 및 표준편차
    """
    df_with_lags = df.copy()
    df_with_lags = df_with_lags.sort_values([building_col, 'date_time'])
    
    # 건물별로 래그 피처 생성
    lag_features = []
    for building_num in df[building_col].unique():
        building_data = df_with_lags[df_with_lags[building_col] == building_num].copy()
        
        if len(building_data) == 0:
            continue
            
        building_data = building_data.sort_values('date_time')
        
        # *** 개선사항 5: 핵심 래그 피처들 ***
        # 1시간 전 (직전 시간)
        building_data['lag_1h'] = building_data[target_col].shift(1)
        
        # 24시간 전 (전날 동시간)  
        building_data['lag_24h'] = building_data[target_col].shift(24)
        
        # 168시간 전 (전주 동시간동요일)
        building_data['lag_168h'] = building_data[target_col].shift(168)
        
        # 롤링 평균 (최근 3시간, 24시간, 168시간)
        building_data['rolling_mean_3h'] = building_data[target_col].rolling(window=3, min_periods=1).mean().shift(1)
        building_data['rolling_mean_24h'] = building_data[target_col].rolling(window=24, min_periods=1).mean().shift(1)
        building_data['rolling_mean_168h'] = building_data[target_col].rolling(window=168, min_periods=1).mean().shift(1)
        
        # 롤링 표준편차
        building_data['rolling_std_24h'] = building_data[target_col].rolling(window=24, min_periods=1).std().shift(1)
        building_data['rolling_std_168h'] = building_data[target_col].rolling(window=168, min_periods=1).std().shift(1)
        
        # 최대/최소값 (최근 24시간)
        building_data['rolling_max_24h'] = building_data[target_col].rolling(window=24, min_periods=1).max().shift(1)
        building_data['rolling_min_24h'] = building_data[target_col].rolling(window=24, min_periods=1).min().shift(1)
        
        # 변화율 피처
        building_data['pct_change_1h'] = building_data[target_col].pct_change(periods=1)
        building_data['pct_change_24h'] = building_data[target_col].pct_change(periods=24)
        
        # 전날 동시간 대비 차이
        building_data['diff_24h'] = building_data[target_col] - building_data['lag_24h']
        
        lag_features.append(building_data)
    
    return pd.concat(lag_features, ignore_index=True)

# 훈련 데이터에 래그 피처 추가 (target이 있는 경우에만)
print("과거 부하 래그 피처 생성 중...")
train_with_lags = create_lag_features(train)

# 테스트 데이터의 경우, 훈련 데이터의 마지막 값들을 이용하여 초기 래그값 설정
def create_test_lag_features(train_df, test_df, target_col='power_consumption', building_col='building_number'):
    """테스트 데이터용 래그 피처 생성"""
    test_with_lags = test_df.copy()
    
    # 각 건물별로 처리
    for building_num in test_df[building_col].unique():
        # 해당 건물의 훈련 데이터 마지막 부분
        train_building = train_df[train_df[building_col] == building_num].copy()
        test_building = test_df[test_df[building_col] == building_num].copy()
        
        if len(train_building) == 0 or len(test_building) == 0:
            continue
            
        # 시간순 정렬
        train_building = train_building.sort_values('date_time')
        test_building = test_building.sort_values('date_time')
        
        # 연결된 시계열 생성 (훈련 데이터 끝 + 테스트 데이터)
        # 테스트 데이터에는 power_consumption이 없으므로 NaN으로 설정
        test_building_temp = test_building.copy()
        test_building_temp[target_col] = np.nan
        
        # 훈련 데이터의 마지막 168개 시점 + 테스트 데이터
        recent_train = train_building.tail(168)
        combined = pd.concat([recent_train, test_building_temp], ignore_index=True)
        combined = combined.sort_values('date_time')
        
        # 래그 피처들을 계산 (NaN인 target에 대해서는 이전 값들로만 계산)
        for i in range(len(recent_train), len(combined)):
            # 1시간 전
            if i >= 1:
                combined.iloc[i, combined.columns.get_loc('lag_1h')] = combined.iloc[i-1][target_col] if not pd.isna(combined.iloc[i-1][target_col]) else recent_train.iloc[-1][target_col]
            
            # 24시간 전  
            if i >= 24:
                combined.iloc[i, combined.columns.get_loc('lag_24h')] = combined.iloc[i-24][target_col]
                
            # 168시간 전
            if i >= 168:
                combined.iloc[i, combined.columns.get_loc('lag_168h')] = combined.iloc[i-168][target_col]
        
        # 테스트 부분만 추출하여 원본에 업데이트
        test_part = combined.iloc[len(recent_train):].copy()
        test_with_lags.loc[test_with_lags[building_col] == building_num, 
                          [col for col in test_part.columns if 'lag_' in col or 'rolling_' in col]] = \
                          test_part[[col for col in test_part.columns if 'lag_' in col or 'rolling_' in col]].values
    
    return test_with_lags


# 실제 래그 피처 생성은 타겟이 없는 테스트 데이터 특성상 간단하게 처리
# 여기서는 기본 통계값으로 대체
for df_name, df in [('train', train_with_lags), ('test', test)]:
    if df_name == 'test':
        # 테스트 데이터는 래그 피처를 0 또는 건물별 평균으로 초기화
        lag_cols = ['lag_1h', 'lag_24h', 'lag_168h', 'rolling_mean_3h', 'rolling_mean_24h', 
                   'rolling_mean_168h', 'rolling_std_24h', 'rolling_std_168h', 
                   'rolling_max_24h', 'rolling_min_24h', 'pct_change_1h', 'pct_change_24h', 'diff_24h']
        
        for col in lag_cols:
            df[col] = 0  # 또는 건물별 평균값으로 대체 가능

# train 데이터를 업데이트
train = train_with_lags

print("래그 피처 생성 완료")
print(f"생성된 래그 피처: {[col for col in train.columns if any(x in col for x in ['lag_', 'rolling_', 'pct_change_', 'diff_'])]}")

In [ ]:
# 셀 6: 개선된 이상치 탐지 및 처리 (가중치 기반)

from scipy import stats
from sklearn.preprocessing import RobustScaler

def detect_outliers_with_weights(df, target_col='power_consumption', building_col='building_number'):
    """
    이상치를 완전 제거하지 않고 가중치를 부여하는 방식
    - 센서 오류/정전: 제거
    - 이벤트성 이상치: 가중치 감소
    """
    df_result = df.copy()
    df_result['sample_weight'] = 1.0  # 기본 가중치
    df_result['is_extreme'] = 0       # 이벤트성 이상치 플래그
    
    outlier_indices_to_remove = []    # 완전 제거할 인덱스
    
    # 건물 유형별 완화 처리 대상
    RELAX_TYPES = ["Public", "Other Buildings", "Hotel and Resort", "Department Store and Outlet"]
    
    for building_num in df[building_col].unique():
        building_mask = df_result[building_col] == building_num
        building_data = df_result[building_mask].copy()
        building_type = building_data['building_type'].iloc[0] if len(building_data) > 0 else "Unknown"
        
        if len(building_data) < 24:  # 데이터가 너무 적으면 스킵
            continue
            
        power_values = building_data[target_col].astype(float)
        
        # 1. 센서 오류/정전 탐지 (완전 제거 대상)
        sensor_errors = []
        
        # 1-1. 연속된 0값 또는 음수값
        zero_negative_mask = (power_values <= 0.01)
        consecutive_zeros = []
        count = 0
        for i, is_zero in enumerate(zero_negative_mask):
            if is_zero:
                count += 1
            else:
                if count >= 4:  # 4시간 이상 연속 0
                    consecutive_zeros.extend(range(i-count, i))
                count = 0
        sensor_errors.extend([building_data.index[i] for i in consecutive_zeros])
        
        # 1-2. 급격한 하락 (전시간 대비 95% 이상 감소)
        prev_values = power_values.shift(1)
        sudden_drop_mask = ((prev_values > 0) & 
                           ((prev_values - power_values) / (prev_values + 1e-9) >= 0.95))
        sensor_errors.extend(building_data.index[sudden_drop_mask])
        
        # 1-3. 황당하게 큰 값 (99.9% 분위수의 5배 초과)
        upper_bound = np.percentile(power_values.dropna(), 99.9) * 5
        extreme_high_mask = power_values > upper_bound
        sensor_errors.extend(building_data.index[extreme_high_mask])
        
        # 2. 이벤트성 이상치 탐지 (가중치 감소 대상)
        # IQR 방법으로 이상치 탐지
        Q1 = power_values.quantile(0.25)
        Q3 = power_values.quantile(0.75)
        IQR = Q3 - Q1
        
        # 건물 유형별 다른 임계값 적용
        if building_type in RELAX_TYPES:
            outlier_factor = 3.0  # 더 관대한 기준
        else:
            outlier_factor = 2.0  # 일반적인 기준
            
        lower_bound = Q1 - outlier_factor * IQR
        upper_bound = Q3 + outlier_factor * IQR
        
        moderate_outliers = building_data.index[
            (power_values < lower_bound) | (power_values > upper_bound)
        ]
        
        # 센서 오류와 겹치지 않는 순수 이벤트성 이상치만 선별
        event_outliers = [idx for idx in moderate_outliers if idx not in sensor_errors]
        
        # 3. 결과 적용
        # 센서 오류는 완전 제거 목록에 추가
        outlier_indices_to_remove.extend(sensor_errors)
        
        # 이벤트성 이상치는 가중치만 감소
        for idx in event_outliers:
            if building_type in RELAX_TYPES:
                df_result.loc[idx, 'sample_weight'] = 0.3  # 관대한 처리
                df_result.loc[idx, 'is_extreme'] = 1
            else:
                df_result.loc[idx, 'sample_weight'] = 0.1  # 강한 페널티
    
    return df_result, list(set(outlier_indices_to_remove))

# 이상치 탐지 및 가중치 부여
print("이상치 탐지 및 가중치 부여 중...")
train_weighted, remove_indices = detect_outliers_with_weights(train)

print(f"센서 오류/정전으로 제거할 샘플: {len(remove_indices)}개")
print(f"가중치 감소 대상 (이벤트성): {sum(train_weighted['sample_weight'] < 1.0)}개")
print(f"극한 이벤트 플래그: {sum(train_weighted['is_extreme'] == 1)}개")

# 센서 오류만 제거하고 나머지는 가중치로 처리
train_cleaned = train_weighted.drop(index=remove_indices).reset_index(drop=True)

# 테스트 데이터에는 기본값 설정
test['sample_weight'] = 1.0
test['is_extreme'] = 0

print(f"최종 훈련 데이터 크기: {train_cleaned.shape}")
print("이상치 처리 완료 (가중치 기반)")

In [ ]:
# 셀 7: 전력 소비 통계 피처 생성 (과거 데이터 기반)

def create_power_statistics_features(df, target_col='power_consumption', building_col='building_number'):
    """전력 소비 패턴 기반 통계 피처 생성"""
    
    # 건물별, 시간대별, 요일별 평균/표준편차
    power_hour_dayofweek_stats = df.groupby([building_col, 'hour', 'day_of_week'])[target_col].agg(['mean', 'std']).reset_index()
    power_hour_dayofweek_stats.columns = [building_col, 'hour', 'day_of_week', 'power_hour_dayofweek_mean', 'power_hour_dayofweek_std']
    
    # 건물별, 시간대별 평균/표준편차  
    power_hour_stats = df.groupby([building_col, 'hour'])[target_col].agg(['mean', 'std', 'min', 'max']).reset_index()
    power_hour_stats.columns = [building_col, 'hour', 'power_hour_mean', 'power_hour_std', 'power_hour_min', 'power_hour_max']
    
    # 건물별, 요일별 평균/표준편차
    power_dayofweek_stats = df.groupby([building_col, 'day_of_week'])[target_col].agg(['mean', 'std']).reset_index()
    power_dayofweek_stats.columns = [building_col, 'day_of_week', 'power_dayofweek_mean', 'power_dayofweek_std']
    
    # 건물별, 월별 평균/표준편차
    power_month_stats = df.groupby([building_col, 'month'])[target_col].agg(['mean', 'std']).reset_index()  
    power_month_stats.columns = [building_col, 'month', 'power_month_mean', 'power_month_std']
    
    # 건물별 전체 통계
    power_building_stats = df.groupby(building_col)[target_col].agg(['mean', 'std', 'min', 'max', 'median']).reset_index()
    power_building_stats.columns = [building_col, 'power_building_mean', 'power_building_std', 'power_building_min', 'power_building_max', 'power_building_median']
    
    return (power_hour_dayofweek_stats, power_hour_stats, power_dayofweek_stats, 
            power_month_stats, power_building_stats)

# 통계 피처 계산
print("전력 소비 통계 피처 생성 중...")
stats_features = create_power_statistics_features(train_cleaned)

power_hour_dayofweek_stats, power_hour_stats, power_dayofweek_stats, power_month_stats, power_building_stats = stats_features

# 훈련 데이터에 통계 피처 병합
train_final = train_cleaned.copy()

train_final = train_final.merge(power_hour_dayofweek_stats, on=['building_number', 'hour', 'day_of_week'], how='left')
train_final = train_final.merge(power_hour_stats, on=['building_number', 'hour'], how='left')
train_final = train_final.merge(power_dayofweek_stats, on=['building_number', 'day_of_week'], how='left')  
train_final = train_final.merge(power_month_stats, on=['building_number', 'month'], how='left')
train_final = train_final.merge(power_building_stats, on='building_number', how='left')

# 테스트 데이터에도 동일한 통계 피처 병합
test_final = test.copy()

test_final = test_final.merge(power_hour_dayofweek_stats, on=['building_number', 'hour', 'day_of_week'], how='left')
test_final = test_final.merge(power_hour_stats, on=['building_number', 'hour'], how='left')
test_final = test_final.merge(power_dayofweek_stats, on=['building_number', 'day_of_week'], how='left')
test_final = test_final.merge(power_month_stats, on=['building_number', 'month'], how='left')
test_final = test_final.merge(power_building_stats, on='building_number', how='left')

# 결측치 처리 (통계 피처의 경우)
stat_columns = [col for col in train_final.columns if 'power_' in col and any(x in col for x in ['_mean', '_std', '_min', '_max', '_median'])]

for col in stat_columns:
    train_final[col] = train_final[col].fillna(train_final[col].median())
    test_final[col] = test_final[col].fillna(train_final[col].median())

# 파생 통계 피처 추가
for df in [train_final, test_final]:
    # 현재 시간대 평균 대비 비율 (train에서만 계산 가능)
    if 'power_consumption' in df.columns:
        df['power_vs_hour_mean_ratio'] = df['power_consumption'] / (df['power_hour_mean'] + 1e-9)
        df['power_vs_building_mean_ratio'] = df['power_consumption'] / (df['power_building_mean'] + 1e-9)
    
    # 시간대별 변동성 지표
    df['power_hour_cv'] = df['power_hour_std'] / (df['power_hour_mean'] + 1e-9)  # 변동계수
    df['power_hour_range'] = df['power_hour_max'] - df['power_hour_min']

print("전력 소비 통계 피처 생성 완료")
print(f"생성된 통계 피처 수: {len(stat_columns)}")
print("추가 파생 통계 피처:", ['power_vs_hour_mean_ratio', 'power_vs_building_mean_ratio', 'power_hour_cv', 'power_hour_range'])

# 데이터 업데이트
train = train_final
test = test_final

In [ ]:
# 셀 8: 시계열 교차검증 및 개선된 모델링

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

# 모델링용 피처 선택
def prepare_model_features(train_df, test_df):
    """모델링용 피처 준비"""
    
    # 제거할 컬럼들
    drop_columns = [
        'solar_power_capacity', 'ess_capacity', 'pcs_capacity',
        'power_consumption', 'rainfall', 'sunshine', 'solar_radiation',
        'hour', 'day', 'month', 'day_of_week', 'date_time', 'day_of_year', 'week_of_year'
    ]
    
    # 존재하는 컬럼만 제거
    drop_columns = [col for col in drop_columns if col in train_df.columns]
    
    X = train_df.drop(columns=drop_columns)
    y = train_df['power_consumption'].astype(float)
    X_test = test_df.drop(columns=[col for col in drop_columns if col in test_df.columns and col != 'power_consumption'])
    
    # 범주형 변수 처리 (원핫 인코딩)
    categorical_columns = ['building_type', 'time_period']
    
    for col in categorical_columns:
        if col in X.columns:
            # 훈련 데이터 원핫 인코딩
            dummies_train = pd.get_dummies(X[col], prefix=col, drop_first=False)
            X = pd.concat([X.drop(columns=[col]), dummies_train], axis=1)
            
            # 테스트 데이터 원핫 인코딩
            dummies_test = pd.get_dummies(X_test[col], prefix=col, drop_first=False)
            X_test = pd.concat([X_test.drop(columns=[col]), dummies_test], axis=1)
    
    # 건물 번호 원핫 인코딩
    X = pd.get_dummies(X, columns=['building_number'], prefix='building', drop_first=False)
    X_test = pd.get_dummies(X_test, columns=['building_number'], prefix='building', drop_first=False)
    
    # 훈련/테스트 데이터 컬럼 맞추기
    X, X_test = X.align(X_test, join='outer', axis=1, fill_value=0)
    
    # NaN/Inf 처리
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    return X, y, X_test

# 피처 준비
print("모델링용 피처 준비 중...")
X, y, X_test = prepare_model_features(train, test)

print(f"최종 피처 수: {X.shape[1]}")
print(f"훈련 데이터 크기: {X.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

# *** 개선사항 6: 시계열 교차검증 (TimeSeriesSplit) ***
def time_series_cross_validation(X, y, sample_weights, n_splits=5):
    """시계열 교차검증을 통한 모델 평가 및 앙상블"""
    
    # 시간순 정렬을 위한 인덱스 생성 (원본 날짜 기준)
    time_index = train.reset_index()['date_time']
    sort_idx = time_index.argsort()
    
    X_sorted = X.iloc[sort_idx].reset_index(drop=True)
    y_sorted = y.iloc[sort_idx].reset_index(drop=True)
    weights_sorted = sample_weights.iloc[sort_idx].reset_index(drop=True)
    
    # TimeSeriesSplit 설정
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=len(X_sorted)//10)
    
    # 모델별 결과 저장
    xgb_scores = []
    lgb_scores = []
    xgb_predictions = []
    lgb_predictions = []
    
    fold_idx = 0
    for train_idx, val_idx in tscv.split(X_sorted):
        fold_idx += 1
        print(f"\n=== Fold {fold_idx} ===")
        
        # 데이터 분할
        X_train_fold, X_val_fold = X_sorted.iloc[train_idx], X_sorted.iloc[val_idx]
        y_train_fold, y_val_fold = y_sorted.iloc[train_idx], y_sorted.iloc[val_idx]
        w_train_fold = weights_sorted.iloc[train_idx]
        
        # 로그 변환 (안전한 변환)
        y_train_log = np.log1p(np.clip(y_train_fold, 0, None))
        y_val_log = np.log1p(np.clip(y_val_fold, 0, None))
        
        # XGBoost 모델
        xgb_model = XGBRegressor(
            learning_rate=0.05,
            n_estimators=1000,
            max_depth=6,
            min_child_weight=3,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=RANDOM_SEED,
            objective=weighted_mse(alpha=1.5),
            tree_method='hist',
            n_jobs=-1
        )
        
        xgb_model.fit(
            X_train_fold, y_train_log,
            sample_weight=w_train_fold,
            eval_set=[(X_val_fold, y_val_log)],
            eval_metric=custom_smape,
            early_stopping_rounds=50,
            verbose=False
        )
        
        # XGBoost 예측
        xgb_pred_log = xgb_model.predict(X_val_fold)
        xgb_pred = np.expm1(xgb_pred_log)
        xgb_score = smape(y_val_fold, xgb_pred)
        xgb_scores.append(xgb_score)
        
        # 테스트 데이터 예측
        xgb_test_pred_log = xgb_model.predict(X_test)
        xgb_test_pred = np.expm1(xgb_test_pred_log)
        xgb_predictions.append(xgb_test_pred)
        
        # LightGBM 모델 
        lgb_model = LGBMRegressor(
            learning_rate=0.05,
            n_estimators=1000,
            max_depth=6,
            min_child_samples=20,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=RANDOM_SEED,
            objective='regression',
            metric='mae',
            n_jobs=-1,
            verbose=-1
        )
        
        lgb_model.fit(
            X_train_fold, y_train_log,
            sample_weight=w_train_fold,
            eval_set=[(X_val_fold, y_val_log)],
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
        )
        
        # LightGBM 예측
        lgb_pred_log = lgb_model.predict(X_val_fold)
        lgb_pred = np.expm1(lgb_pred_log)
        lgb_score = smape(y_val_fold, lgb_pred)
        lgb_scores.append(lgb_score)
        
        # 테스트 데이터 예측
        lgb_test_pred_log = lgb_model.predict(X_test)
        lgb_test_pred = np.expm1(lgb_test_pred_log)
        lgb_predictions.append(lgb_test_pred)
        
        print(f"Fold {fold_idx} - XGB SMAPE: {xgb_score:.4f}, LGB SMAPE: {lgb_score:.4f}")
    
    # 평균 성능
    avg_xgb_score = np.mean(xgb_scores)
    avg_lgb_score = np.mean(lgb_scores)
    
    print(f"\n=== 최종 교차검증 결과 ===")
    print(f"XGBoost 평균 SMAPE: {avg_xgb_score:.4f} (±{np.std(xgb_scores):.4f})")
    print(f"LightGBM 평균 SMAPE: {avg_lgb_score:.4f} (±{np.std(lgb_scores):.4f})")
    
    # *** 개선사항 7: 앙상블 (중앙값 사용) ***
    xgb_ensemble = np.median(xgb_predictions, axis=0)
    lgb_ensemble = np.median(lgb_predictions, axis=0)
    
    # 최종 앙상블 (XGBoost + LightGBM 중앙값)
    final_ensemble = np.median([xgb_ensemble, lgb_ensemble], axis=0)
    
    return final_ensemble, avg_xgb_score, avg_lgb_score

# 시계열 교차검증 실행
print("\n시계열 교차검증 및 앙상블 모델링 시작...")
sample_weights = train['sample_weight']

final_predictions, xgb_score, lgb_score = time_series_cross_validation(X, y, sample_weights, n_splits=5)

print(f"\n최종 앙상블 완료")
print(f"예측 결과 통계:")
print(f"  - 최소값: {final_predictions.min():.2f}")
print(f"  - 최대값: {final_predictions.max():.2f}")
print(f"  - 평균값: {final_predictions.mean():.2f}")
print(f"  - 중앙값: {np.median(final_predictions):.2f}")

In [ ]:
# 셀 9: 결과 저장 및 검증

# 결과 파일 생성
submission = pd.read_csv('sample_submission.csv')
submission['answer'] = final_predictions

# 예측값 검증
print("=== 예측 결과 검증 ===")
print(f"총 예측 샘플 수: {len(final_predictions)}")
print(f"음수 예측값: {sum(final_predictions < 0)}개")
print(f"0 예측값: {sum(final_predictions == 0)}개")
print(f"이상 큰 값 (99.9% 분위수 초과): {sum(final_predictions > np.percentile(final_predictions, 99.9) * 3)}개")

# 음수값이 있다면 0으로 클리핑
if sum(final_predictions < 0) > 0:
    print("음수 예측값을 0으로 클리핑합니다.")
    final_predictions = np.clip(final_predictions, 0, None)
    submission['answer'] = final_predictions

# 건물별 예측 통계
print("\n=== 건물 유형별 예측 통계 ===")
test_with_predictions = test.copy()
test_with_predictions['predictions'] = final_predictions

building_type_stats = test_with_predictions.groupby('building_type')['predictions'].agg([
    'count', 'mean', 'std', 'min', 'max', 'median'
]).round(2)

print(building_type_stats)

# 시간대별 예측 패턴 검증
print("\n=== 시간대별 예측 패턴 ===")
test_with_predictions['hour'] = test_with_predictions['date_time'].dt.hour
hourly_stats = test_with_predictions.groupby('hour')['predictions'].agg(['mean', 'std']).round(2)
print("시간별 평균 전력 소비량 (상위 5개/하위 5개 시간대):")
print("상위 5개:", hourly_stats.sort_values('mean', ascending=False).head())
print("하위 5개:", hourly_stats.sort_values('mean', ascending=True).head())

# 요일별 예측 패턴 검증
print("\n=== 요일별 예측 패턴 ===")
test_with_predictions['day_of_week'] = test_with_predictions['date_time'].dt.dayofweek
weekday_stats = test_with_predictions.groupby('day_of_week')['predictions'].agg(['mean', 'std']).round(2)
weekday_names = ['월', '화', '수', '목', '금', '토', '일']
weekday_stats.index = weekday_names
print(weekday_stats)

# 최종 파일 저장
output_filename = f'submission_enhanced_ensemble_{pd.Timestamp.now().strftime("%Y%m%d_%H%M")}.csv'
submission.to_csv(output_filename, index=False)

print(f"\n=== 최종 결과 ===")
print(f"제출 파일 저장: {output_filename}")
print(f"XGBoost 평균 SMAPE: {xgb_score:.4f}")
print(f"LightGBM 평균 SMAPE: {lgb_score:.4f}")
print(f"앙상블 모델 완성!")